In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

def kinematic_rk4(t, f, r0, v0):
    """
    Simulates the motion of an object using the 4th-order Runge-Kutta (RK4) method
    for systems described by position, velocity, and a time-dependent acceleration function.

    Parameters
    ----------
    t : ndarray
        1D array of time points at which to compute the solution. Assumes uniform spacing.
    f : callable
        Function of the form `f(t, r, v)` returning the acceleration (a vector) at time `t`,
        position `r`, and velocity `v`.
    r0 : ndarray
        Initial position vector.
    v0 : ndarray
        Initial velocity vector.

    Returns
    -------
    r : ndarray
        Array of positions at each time step. Shape: (len(t), len(r0)).
    v : ndarray
        Array of velocities at each time step. Shape: (len(t), len(v0)).
    a : ndarray
        Array of accelerations at each time step. Shape: (len(t), len(v0)).
    """
    dt = t[1] - t[0]  # Assuming uniform time steps
    half_dt = dt / 2
    dim = len(r0)

    r = np.zeros((len(t), dim))
    v = np.zeros((len(t), dim))
    a = np.zeros((len(t), dim))

    r[0] = r0
    v[0] = v0
    a[0] = f(t[0], r0, v0)

    for i in range(len(t) - 1):
        t_i = t[i]
        r_i = r[i]
        v_i = v[i]

        k1_v = f(t_i, r_i, v_i)
        k2_v = f(t_i + half_dt, r_i + v_i * half_dt, v_i + k1_v * half_dt)
        k3_v = f(t_i + half_dt, r_i + v_i * half_dt, v_i + k2_v * half_dt)
        k4_v = f(t_i + dt, r_i + v_i * dt, v_i + k3_v * dt)

        k1_r = v_i
        k2_r = v_i + k1_v * half_dt
        k3_r = v_i + k2_v * half_dt
        k4_r = v_i + k3_v * dt

        v[i + 1] = v_i + (k1_v + 2 * k2_v + 2 * k3_v + k4_v) * dt / 6
        r[i + 1] = r_i + (k1_r + 2 * k2_r + 2 * k3_r + k4_r) * dt / 6
        a[i + 1] = f(t[i + 1], r[i + 1], v[i + 1])
        
    return r, v, a


: 

In [ ]:

def normalize (vec):
    if np.linalg.norm(vec) == 0:
        return vec*0
    else:
        return vec / np.linalg.norm(vec)

def vxcl(v1, v2):
    """
    Returns the components of v2 projected onto a plane normal to v1.
    
    Parameters:
    v1 : array-like - Normal vector defining the plane
    v2 : array-like - Vector to be projected
    
    Returns:
    v2_proj : ndarray - Projection of v2 onto the plane normal to v1
    """
    v1 = np.asarray(v1, dtype=float)
    v2 = np.asarray(v2, dtype=float)
    
    # Compute the projection of v2 along v1
    proj_scalar = np.dot(v2, v1) / np.dot(v1, v1)
    v2_parallel = proj_scalar * v1
    
    # Subtract to get the perpendicular component (projection onto plane)
    v2_perpendicular = v2 - v2_parallel
    
    return v2_perpendicular  


# Orbital parameters
mu = 1
a_init = 14 
e_init = 0.5 
p_init = a_init * (1 - e_init**2)

# Full orbit (for reference)
theta = np.linspace(0, 2 * np.pi, 1000)
r_init_mag = p_init / (1 + e_init * np.cos(theta))
r_init = r_init_mag * np.array([np.cos(theta), np.sin(theta)])

# Point of interest (true anomaly = 45°)
theta_pt = np.deg2rad(200)
r_pt = p_init / (1 + e_init * np.cos(theta_pt)) * np.array([np.cos(theta_pt), np.sin(theta_pt)])

# Velocity vector at the point (derived from orbital mechanics)
v_init = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt), e_init + np.cos(theta_pt)])


R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.3f}")
theta_targ = np.linspace(0, 2 * np.pi, 1000)

##  CHANGE PERIAPSIS
# Target parameters calculation
r_pf =2
e_targ = (r_pf - R)/(R * np.cos(theta_pt) - r_pf)
print(f"e targ : {e_targ:.3f}")
if e_targ < 0:
    p_targ = 0
    print("NO ORBIT POSSIBLE")
elif e_targ == 1:
    p_targ = 2*r_pf
elif e_targ > 1:
    theta_targ = np.linspace(0,np.arccos(- 1/ e_targ)-0.01,500) + np.linspace(2*np.pi - np.arccos(- 1/ e_targ)+ 0.01,2*np.pi,500)
    a_targ = r_pf/(1 - e_targ)
    p_targ = a_targ * (1 - e_targ**2)
else:
    a_targ = r_pf/(1 - e_targ)
    p_targ = a_targ * (1 - e_targ**2)

# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta_targ))) * np.array([np.cos(theta_targ), np.sin(theta_targ)])
if e_targ < 0:
    v_targ = v_init
else : v_targ = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt), e_targ + np.cos(theta_pt)])

delta_v = v_targ - v_init

v_prograde = normalize(v_init)
v_normal   = normalize(vxcl(v_init,r_pt))
delta_node_v = np.array([np.dot(delta_v,v_prograde),np.dot(delta_v,v_normal)])
print(delta_node_v)
# Plotting
plt.figure(figsize=(8, 8))

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)

# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

# Draw position vector (r_vec: from origin to r_pt)
plt.arrow(0, 0, r_pt[0], r_pt[1], 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
plt.arrow(r_pt[0], r_pt[1], v_init[0] * scale, v_init[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_targ[0] * scale, v_targ[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt[0], r_pt[1], delta_v[0] * scale, delta_v[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
unit_scale = 0.2
plt.arrow(r_pt[0], r_pt[1], v_prograde[0] * unit_scale, v_prograde[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_normal[0] * unit_scale, v_normal[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Customize plot
plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-15,15])
plt.ylim([-15,15])
plt.show()

In [ ]:
## CHANGE APOAPSIS
# Point of interest (true anomaly = 45°)
theta_pt = np.deg2rad(65)
r_pt = p_init / (1 + e_init * np.cos(theta_pt)) * np.array([np.cos(theta_pt), np.sin(theta_pt)])

# Velocity vector at the poin
v_init = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt), e_init + np.cos(theta_pt)])


R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.3f}")
theta_targ = np.linspace(0, 2 * np.pi, 1000)
# Target parameters calculation
r_af = 2
e_targ = (r_af - R)/(R * np.cos(theta_pt) + r_af)

print(f"e targ : {e_targ:.3f}")
if e_targ < 0 or e_targ >= 1:
    p_targ = 0
    print("NO ORBIT POSSIBLE")
else:
    a_targ = r_af/(1 + e_targ)
    p_targ = a_targ * (1 - e_targ**2)

# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta_targ))) * np.array([np.cos(theta_targ), np.sin(theta_targ)])
if e_targ < 0 or e_targ >= 1:
    v_targ = v_init
else : v_targ = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt), e_targ + np.cos(theta_pt)])

delta_v = v_targ - v_init

v_prograde = normalize(v_init)
v_normal   = normalize(vxcl(v_init,r_pt))
delta_node_v = np.array([np.dot(delta_v,v_prograde),np.dot(delta_v,v_normal)])
print(delta_node_v)

# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
plt.arrow(r_pt[0], r_pt[1], v_init[0] * scale, v_init[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_targ[0] * scale, v_targ[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt[0], r_pt[1], delta_v[0] * scale, delta_v[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
unit_scale = 0.2
plt.arrow(r_pt[0], r_pt[1], v_prograde[0] * unit_scale, v_prograde[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_normal[0] * unit_scale, v_normal[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()

In [ ]:
## CHANGE E
# Point of interest (true anomaly)
theta_pt = np.deg2rad(170)
r_pt = p_init / (1 + e_init * np.cos(theta_pt)) * np.array([np.cos(theta_pt), np.sin(theta_pt)])

# Velocity vector at the point
v_init = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt), e_init + np.cos(theta_pt)])


R = np.linalg.norm(r_pt)  # Magnitude of r_pt
e_targ = 1.5
print(f"Rmag : {R:.3f}")
if e_targ < 0:
    print("NO ORBIT POSSIBLE (e < 0)")
    p_targ = 0
elif e_targ == 1:  # Parabolic case
    p_targ = R * (1 + np.cos(theta_pt))  # From r = p/(1 + cosθ) at θ=θ_pt
elif e_targ > 1:  # Hyperbolic case
    a_targ = R * (1 + e_targ * np.cos(theta_pt)) / (1 - e_targ**2)  # Negative
    p_targ = a_targ * (1 - e_targ**2)  # Positive
     # Check if the focus flips to the right side
    if np.cos(theta_pt) < -1 / e_targ:
        print("NO ORBIT POSSIBLE (Hyperbola flips focus)")
        e_targ = -1
else:  # 0 ≤ e_targ < 1 (Elliptical case)
    a_targ = R * (1 + e_targ * np.cos(theta_pt)) / (1 - e_targ**2)  # Positive
    p_targ = a_targ * (1 - e_targ**2)  # Positive

print(a_targ * (1 - e_targ), a_targ * (1 + e_targ))
# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta_targ))) * np.array([np.cos(theta_targ), np.sin(theta_targ)])
if e_targ < 0:
    v_targ = v_init
else : v_targ = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt), e_targ + np.cos(theta_pt)])

delta_v = v_targ - v_init
print ( f"dV mag = {np.linalg.norm(delta_v)}")
v_prograde = normalize(v_init)
v_normal   = normalize(vxcl(v_init,r_pt))
delta_node_v = np.array([np.dot(delta_v,v_prograde),np.dot(delta_v,v_normal)])
print(delta_node_v)

# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
plt.arrow(r_pt[0], r_pt[1], v_init[0] * scale, v_init[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_targ[0] * scale, v_targ[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt[0], r_pt[1], delta_v[0] * scale, delta_v[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
unit_scale = 0.2
plt.arrow(r_pt[0], r_pt[1], v_prograde[0] * unit_scale, v_prograde[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_normal[0] * unit_scale, v_normal[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()

In [ ]:
## CHANGE SMJA
# Point of interest (true anomaly)
theta_pt = np.deg2rad(100)
r_pt = p_init / (1 + e_init * np.cos(theta_pt)) * np.array([np.cos(theta_pt), np.sin(theta_pt)])

# Velocity vector at the point
v_init = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt), e_init + np.cos(theta_pt)])


R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.10f}")
a_targ = 1.6

discriminant = (R * np.cos(theta_pt))**2 - 4 * a_targ * (R - a_targ)

if discriminant < 0:
    print("NO ORBIT POSSIBLE (No real eccentricity)")
    e_targ = -1  # Force failure
else:
    e_targ = (-R * np.cos(theta_pt) + np.sqrt(discriminant)) / (2 * a_targ)
# Fail condition 2: Eccentricity bounds
if e_targ < 0 or e_targ >= 1:  # Note: e_targ=1 is parabolic (rejected)
    print("NO ORBIT POSSIBLE (Invalid eccentricity)")
    p_targ = 0
else:
    p_targ = a_targ * (1 - e_targ**2)

print(discriminant)
print(f"e = {e_targ}")

# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta_targ))) * np.array([np.cos(theta_targ), np.sin(theta_targ)])
if a_targ == a_init:
    print("NO CHANGE")
if e_targ < 0 or e_targ > 1:
    v_targ = v_init
else : 
    v_targ = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt), e_targ + np.cos(theta_pt)])
delta_v = v_targ - v_init

v_prograde = normalize(v_init)
v_normal   = normalize(vxcl(v_init,r_pt))
delta_node_v = np.array([np.dot(delta_v,v_prograde),np.dot(delta_v,v_normal)])
print(delta_node_v)

# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
plt.arrow(r_pt[0], r_pt[1], v_init[0] * scale, v_init[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_targ[0] * scale, v_targ[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt[0], r_pt[1], delta_v[0] * scale, delta_v[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
unit_scale = 0.2
plt.arrow(r_pt[0], r_pt[1], v_prograde[0] * unit_scale, v_prograde[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt[0], r_pt[1], v_normal[0] * unit_scale, v_normal[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()

In [ ]:
## change pe and ap 


a_targ  = 1.5
e_targ  = 0
p_targ  = a_targ * (1 - e_targ**2)
print(a_targ, e_targ, p_targ)
# Point of interest (true anomaly)
angle_parameter = (p_targ - p_init) / (p_init * e_targ - p_targ * e_init)
if angle_parameter < 1 or angle_parameter > 1:
    print ( "NO ORBIT POSSIBLE")
print(angle_parameter)
theta_pt1 = np.arccos( angle_parameter )
theta_pt2 =  - np.arccos( angle_parameter) + 2 * np.pi

r_pt1 = p_init / (1 + e_init * np.cos(theta_pt1)) * np.array([np.cos(theta_pt1), np.sin(theta_pt1)])
r_pt2 = p_init / (1 + e_init * np.cos(theta_pt2)) * np.array([np.cos(theta_pt2), np.sin(theta_pt2)])

# Velocity vector at the point
v_init1 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt1), e_init + np.cos(theta_pt1)])
v_init2 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt2), e_init + np.cos(theta_pt2)])
R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.3f}")
    
# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta))) * np.array([np.cos(theta), np.sin(theta)])
v_targ1 = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt1), e_targ + np.cos(theta_pt1)])
v_targ2 = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt2), e_targ + np.cos(theta_pt2)])

delta_v1 = v_targ1 - v_init1
delta_v2 = v_targ2 - v_init2

v_prograde1 = normalize(v_init1)
v_normal1   = normalize(vxcl(v_init1,r_pt1))
delta_node_v1 = np.array([np.dot(delta_v1,v_prograde1),np.dot(delta_v1,v_normal1)])
print(delta_node_v1)
v_prograde2 = normalize(v_init2)
v_normal2   = normalize(vxcl(v_init2,r_pt2))
delta_node_v2 = np.array([np.dot(delta_v2,v_prograde2),np.dot(delta_v2,v_normal2)])
print(delta_node_v2)

# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
unit_scale = 0.2
plt.arrow(r_pt1[0], r_pt1[1], v_init1[0] * scale, v_init1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_targ1[0] * scale, v_targ1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], delta_v1[0] * scale, delta_v1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_prograde1[0] * unit_scale, v_prograde1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_normal1[0] * unit_scale, v_normal1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

plt.arrow(r_pt2[0], r_pt2[1], v_init2[0] * scale, v_init2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_targ2[0] * scale, v_targ2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], delta_v2[0] * scale, delta_v2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_prograde2[0] * unit_scale, v_prograde2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_normal2[0] * unit_scale, v_normal2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()

In [ ]:
## change pe and ap 

pe_targ = 0.9
ap_targ = 3.5

a_targ  = (pe_targ + ap_targ) / 2
e_targ  = (ap_targ - pe_targ) / (ap_targ + pe_targ)
p_targ  = a_targ * (1 - e_targ**2)
print(a_targ, e_targ, p_targ)

# Point of interest (true anomaly)
angle_parameter = (p_targ - p_init) / (p_init * e_targ - p_targ * e_init)
if angle_parameter < 1 or angle_parameter > 1:
    print ( "NO ORBIT POSSIBLE")
print(angle_parameter)
theta_pt1 = np.arccos( angle_parameter )
theta_pt2 =  - np.arccos( angle_parameter) + 2 * np.pi

r_pt1 = p_init / (1 + e_init * np.cos(theta_pt1)) * np.array([np.cos(theta_pt1), np.sin(theta_pt1)])
r_pt2 = p_init / (1 + e_init * np.cos(theta_pt2)) * np.array([np.cos(theta_pt2), np.sin(theta_pt2)])

# Velocity vector at the point
v_init1 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt1), e_init + np.cos(theta_pt1)])
v_init2 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt2), e_init + np.cos(theta_pt2)])
R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.3f}")
    
# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta))) * np.array([np.cos(theta), np.sin(theta)])
v_targ1 = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt1), e_targ + np.cos(theta_pt1)])
v_targ2 = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt2), e_targ + np.cos(theta_pt2)])

delta_v1 = v_targ1 - v_init1
delta_v2 = v_targ2 - v_init2

v_prograde1 = normalize(v_init1)
v_normal1   = normalize(vxcl(v_init1,r_pt1))
delta_node_v1 = np.array([np.dot(delta_v1,v_prograde1),np.dot(delta_v1,v_normal1)])
print(delta_node_v1)
v_prograde2 = normalize(v_init2)
v_normal2   = normalize(vxcl(v_init2,r_pt2))
delta_node_v2 = np.array([np.dot(delta_v2,v_prograde2),np.dot(delta_v2,v_normal2)])
print(delta_node_v2)

# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
unit_scale = 0.2
plt.arrow(r_pt1[0], r_pt1[1], v_init1[0] * scale, v_init1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_targ1[0] * scale, v_targ1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], delta_v1[0] * scale, delta_v1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_prograde1[0] * unit_scale, v_prograde1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_normal1[0] * unit_scale, v_normal1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

plt.arrow(r_pt2[0], r_pt2[1], v_init2[0] * scale, v_init2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_targ2[0] * scale, v_targ2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], delta_v2[0] * scale, delta_v2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_prograde2[0] * unit_scale, v_prograde2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_normal2[0] * unit_scale, v_normal2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()

In [ ]:
## CHANGE ARGPERI
# Point of interest (true anomaly)
psi_init = np.deg2rad(0)
# Full orbit (for reference)
theta = np.linspace(0, 2 * np.pi, 1000)
r_init_mag = p_init / (1 + e_init * np.cos(theta))
r_init = r_init_mag * np.array([np.cos(theta + psi_init), np.sin(theta + psi_init)])



R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.3f}")

psi_targ = np.deg2rad(70) 

# Corrected target orbit calculation
r_targ = (p_init/(1 + e_init * np.cos(theta))) * np.array([np.cos(theta + psi_targ), np.sin(theta + psi_targ)])

theta_pt1 = ((psi_init + psi_targ)/2 - psi_init)
theta_targ1 = ((psi_init + psi_targ)/2 - psi_targ)
r_pt1 = p_init / (1 + e_init * np.cos(theta_pt1)) * np.array([np.cos(theta_pt1 + psi_init), np.sin(theta_pt1 + psi_init)])
# Velocity vector at the point
v_init1 =     np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt1 + psi_init) - e_init * np.sin(psi_init), e_init * np.cos(psi_init) + np.cos(theta_pt1 + psi_init)])
#v_init_arr = np.sqrt(mu / p_init) * np.array([-np.sin(theta + psi_init) - e_init * np.sin(psi_init), e_init * np.cos(psi_init) + np.cos(theta + psi_init)])
v_targ1 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_targ1 + psi_targ) - e_init * np.sin(psi_targ), e_init * np.cos(psi_targ) + np.cos(theta_targ1 + psi_targ)])
delta_v1 = v_targ1 - v_init1
v_prograde1 = normalize(v_init1)
v_normal1   = normalize(vxcl(v_init1,r_pt1))
delta_node_v1 = np.array([np.dot(delta_v1,v_prograde1),np.dot(delta_v1,v_normal1)])
print(delta_node_v1, np.linalg.norm(delta_v1))


theta_pt2 = ((psi_init + psi_targ)/2 - psi_init) + np.pi
theta_targ2 = ((psi_init + psi_targ)/2 - psi_targ) + np.pi
r_pt2 = p_init / (1 + e_init * np.cos(theta_pt2)) * np.array([np.cos(theta_pt2 + psi_init), np.sin(theta_pt2 + psi_init)])
# Velocity vector at the point
v_init2 =     np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt2 + psi_init) - e_init * np.sin(psi_init), e_init * np.cos(psi_init) + np.cos(theta_pt2 + psi_init)])
v_init_arr = np.sqrt(mu / p_init) * np.array([-np.sin(theta + psi_init) - e_init * np.sin(psi_init), e_init * np.cos(psi_init) + np.cos(theta + psi_init)])
v_targ2 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_targ2 + psi_targ) - e_init * np.sin(psi_targ), e_init * np.cos(psi_targ) + np.cos(theta_targ2 + psi_targ)])
delta_v2 = v_targ2 - v_init2
v_prograde2 = normalize(v_init2)
v_normal2   = normalize(vxcl(v_init2,r_pt1))
delta_node_v2 = np.array([np.dot(delta_v2,v_prograde2),np.dot(delta_v2,v_normal2)])
print(delta_node_v2, np.linalg.norm(delta_v2))


# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")
plt.plot([0, a_init * (1 - e_init) * np.cos(psi_targ)],[0,a_init * (1 - e_init) * np.sin(psi_targ)],"green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
plt.arrow(r_pt1[0], r_pt1[1], v_init1[0] * scale, v_init1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_targ1[0] * scale, v_targ1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], delta_v1[0] * scale, delta_v1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
unit_scale = 0.2
plt.arrow(r_pt1[0], r_pt1[1], v_prograde1[0] * unit_scale, v_prograde1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_normal1[0] * unit_scale, v_normal1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

plt.arrow(r_pt2[0], r_pt2[1], v_init2[0] * scale, v_init2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_targ2[0] * scale, v_targ2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], delta_v2[0] * scale, delta_v2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_prograde2[0] * unit_scale, v_prograde2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_normal2[0] * unit_scale, v_normal2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# periapsis marker
plt.plot([0, a_init * (1 - e_init) * np.cos(psi_init)],[0,a_init * (1 - e_init) * np.sin(psi_init)],"blue")
# Plot full orbit
# plt.plot(v_init_arr[0,:], v_init_arr[1,:], "red", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt1[0], r_pt1[1], color='black', s=50, zorder=3)

midway_ang = (psi_init + psi_targ)/2
plt.plot([0,np.cos(midway_ang)],[0,np.sin(midway_ang)])
plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()


In [ ]:
## CHANGE LAN 

pe_targ = 0.9
ap_targ = 3.5

a_targ  = (pe_targ + ap_targ) / 2
e_targ  = (ap_targ - pe_targ) / (ap_targ + pe_targ)
p_targ  = a_targ * (1 - e_targ**2)
print(a_targ, e_targ, p_targ)

# Point of interest (true anomaly)
angle_parameter = (p_targ - p_init) / (p_init * e_targ - p_targ * e_init)
if angle_parameter < 1 or angle_parameter > 1:
    print ( "NO ORBIT POSSIBLE")
print(angle_parameter)
theta_pt1 = np.arccos( angle_parameter )
theta_pt2 =  - np.arccos( angle_parameter) + 2 * np.pi

r_pt1 = p_init / (1 + e_init * np.cos(theta_pt1)) * np.array([np.cos(theta_pt1), np.sin(theta_pt1)])
r_pt2 = p_init / (1 + e_init * np.cos(theta_pt2)) * np.array([np.cos(theta_pt2), np.sin(theta_pt2)])

# Velocity vector at the point
v_init1 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt1), e_init + np.cos(theta_pt1)])
v_init2 = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt2), e_init + np.cos(theta_pt2)])
R = np.linalg.norm(r_pt)  # Magnitude of r_pt
print(f"Rmag : {R:.3f}")
    
# Corrected target orbit calculation
r_targ = (p_targ/(1 + e_targ * np.cos(theta))) * np.array([np.cos(theta), np.sin(theta)])
v_targ1 = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt1), e_targ + np.cos(theta_pt1)])
v_targ2 = np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt2), e_targ + np.cos(theta_pt2)])

delta_v1 = v_targ1 - v_init1
delta_v2 = v_targ2 - v_init2

v_prograde1 = normalize(v_init1)
v_normal1   = normalize(vxcl(v_init1,r_pt1))
delta_node_v1 = np.array([np.dot(delta_v1,v_prograde1),np.dot(delta_v1,v_normal1)])
print(delta_node_v1)
v_prograde2 = normalize(v_init2)
v_normal2   = normalize(vxcl(v_init2,r_pt2))
delta_node_v2 = np.array([np.dot(delta_v2,v_prograde2),np.dot(delta_v2,v_normal2)])
print(delta_node_v2)

# Plotting
plt.figure(figsize=(8, 8))

## plot new orbit
plt.plot(r_targ[0,:], r_targ[1,:], "green")

# Draw velocity vector 
scale = 1.0  # Adjust for better visibility
unit_scale = 0.2
plt.arrow(r_pt1[0], r_pt1[1], v_init1[0] * scale, v_init1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_targ1[0] * scale, v_targ1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], delta_v1[0] * scale, delta_v1[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_prograde1[0] * unit_scale, v_prograde1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt1[0], r_pt1[1], v_normal1[0] * unit_scale, v_normal1[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

plt.arrow(r_pt2[0], r_pt2[1], v_init2[0] * scale, v_init2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_targ2[0] * scale, v_targ2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='green', ec='green', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], delta_v2[0] * scale, delta_v2[1] * scale, 
          head_width=0.05, head_length=0.1, 
          fc='red', ec='red', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_prograde2[0] * unit_scale, v_prograde2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)
plt.arrow(r_pt2[0], r_pt2[1], v_normal2[0] * unit_scale, v_normal2[1] * unit_scale, 
          head_width=0.05, head_length=0.1, 
          fc='cyan', ec='cyan', zorder=4)

# Plot full orbit
plt.plot(r_init[0,:], r_init[1,:], "blue", alpha=1, zorder=2)
# Plot central body (focus) and the point
plt.scatter(0, 0, color='yellow', s=50, zorder=10)
plt.scatter(r_pt[0], r_pt[1], color='black', s=50, zorder=3)

plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Constants
mu = 1.0  # Standard gravitational parameter
p_init = 1.0  # Initial semi-latus rectum
e_init = 0.5  # Initial eccentricity

# Target orbit parameters
pe_targ = 0.9  # Periapsis
ap_targ = 3.5  # Apoapsis
a_targ = (pe_targ + ap_targ) / 2
e_targ = (ap_targ - pe_targ) / (ap_targ + pe_targ)
p_targ = a_targ * (1 - e_targ**2)

# 3D Orbital Elements
i_targ = np.deg2rad(30)  # Inclination (radians)
lan_targ = np.deg2rad(45)  # RAAN (radians)
argperi_targ = np.deg2rad(60)  # Argument of periapsis (radians)

# Find true anomaly (theta) where orbits intersect
angle_parameter = (p_targ - p_init) / (p_init * e_targ - p_targ * e_init)
if angle_parameter < -1 or angle_parameter > 1:
    print("NO ORBIT POSSIBLE (No valid theta)")
theta_pt1 = np.arccos(angle_parameter)
theta_pt2 = -theta_pt1 + 2 * np.pi

# Position and velocity in 2D (for intersection points)
r_pt1_2d = p_init / (1 + e_init * np.cos(theta_pt1)) * np.array([np.cos(theta_pt1), np.sin(theta_pt1), 0])
r_pt2_2d = p_init / (1 + e_init * np.cos(theta_pt2)) * np.array([np.cos(theta_pt2), np.sin(theta_pt2), 0])
v_init1_2d = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt1), e_init + np.cos(theta_pt1), 0])
v_init2_2d = np.sqrt(mu / p_init) * np.array([-np.sin(theta_pt2), e_init + np.cos(theta_pt2), 0])

# --- Convert to 3D ---
def rotate_3d(vec, i, lan, argperi):
    """Rotate a vector by inclination (i), RAAN (lan), and argument of periapsis (argperi)."""
    # Rotation matrices
    R_lan = np.array([
        [np.cos(lan), -np.sin(lan), 0],
        [np.sin(lan), np.cos(lan), 0],
        [0, 0, 1]
    ])
    R_i = np.array([
        [1, 0, 0],
        [0, np.cos(i), -np.sin(i)],
        [0, np.sin(i), np.cos(i)]
    ])
    R_argperi = np.array([
        [np.cos(argperi), -np.sin(argperi), 0],
        [np.sin(argperi), np.cos(argperi), 0],
        [0, 0, 1]
    ])
    # Apply rotations
    return R_lan @ R_i @ R_argperi @ vec

# Rotate position and velocity vectors
r_pt1 = rotate_3d(r_pt1_2d, i_targ, lan_targ, argperi_targ)
r_pt2 = rotate_3d(r_pt2_2d, i_targ, lan_targ, argperi_targ)
v_init1 = rotate_3d(v_init1_2d, i_targ, lan_targ, argperi_targ)
v_init2 = rotate_3d(v_init2_2d, i_targ, lan_targ, argperi_targ)

# Target orbit velocity (3D)
v_targ1 = rotate_3d(
    np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt1), e_targ + np.cos(theta_pt1), 0]),
    i_targ, lan_targ, argperi_targ
)
v_targ2 = rotate_3d(
    np.sqrt(mu / p_targ) * np.array([-np.sin(theta_pt2), e_targ + np.cos(theta_pt2), 0]),
    i_targ, lan_targ, argperi_targ
)

# Delta-V calculation
delta_v1 = v_targ1 - v_init1
delta_v2 = v_targ2 - v_init2

# --- Plotting ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Generate full orbits (3D)
theta = np.linspace(0, 2 * np.pi, 100)
r_init_2d = (p_init / (1 + e_init * np.cos(theta)) * np.array([np.cos(theta), np.sin(theta), np.zeros_like(theta)]))
r_targ_2d = (p_targ / (1 + e_targ * np.cos(theta))) * np.array([np.cos(theta), np.sin(theta), np.zeros_like(theta)])

# Rotate full orbits to 3D
r_init = np.array([rotate_3d(r_init_2d[:, i], i_targ, lan_targ, argperi_targ) for i in range(len(theta))]).T
r_targ = np.array([rotate_3d(r_targ_2d[:, i], i_targ, lan_targ, argperi_targ) for i in range(len(theta))]).T

# Plot orbits
ax.plot(r_init[0], r_init[1], r_init[2], 'b', label="Initial Orbit", alpha=0.7)
ax.plot(r_targ[0], r_targ[1], r_targ[2], 'g', label="Target Orbit", alpha=0.7)

# Plot points and velocity vectors
scale = 0.5
ax.quiver(r_pt1[0], r_pt1[1], r_pt1[2], v_init1[0], v_init1[1], v_init1[2], color='blue', label="Initial Velocity", length=scale)
ax.quiver(r_pt1[0], r_pt1[1], r_pt1[2], v_targ1[0], v_targ1[1], v_targ1[2], color='green', label="Target Velocity", length=scale)
ax.quiver(r_pt1[0], r_pt1[1], r_pt1[2], delta_v1[0], delta_v1[1], delta_v1[2], color='red', label="Delta-V", length=scale)

# Central body
ax.scatter(0, 0, 0, color='yellow', s=100, label="Central Body")

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.title("3D Orbital Maneuver with Inclination, RAAN, and Argument of Periapsis")
plt.show()

In [ ]:
def g_acc(t,r,v):
    return - (mu / np.linalg.norm(r)**3) * r

t_period = 2*np.pi*np.sqrt(a_init**3/ mu)
t_sim = np.linspace(0, t_period, 1000)
r_sim, v_sim , a_sim = kinematic_rk4(t_sim,g_acc,r_pt,v_init)

plt.plot(r_sim[:,0],r_sim[:,1],"blue")

plt.arrow(0, 0, r_pt[0], r_pt[1], 
          head_width=0.05, head_length=0.1, 
          fc='blue', ec='blue', zorder=4, 
          label="Position Vector (r)")

# Plot central body (focus) and the point
plt.scatter(0, 0, color='red', s=50, zorder=3, label="Central Body")
plt.gca().set_aspect("equal")
plt.grid(True, zorder=1)
plt.xlabel("x")
plt.ylabel("y")

In [ ]:
## APOAPSIS CASE 
## e_targ = r_a - R / Rcostheta + r
## a_tart = r_a / (1 + e_targ)


## SMJA 

## ECCENTRICITY

## ARGPERI

